In [ ]:
import numpy as np
import signalflow as sf
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Create app
app = App()

In [2]:
# load an image
img_path = "images/cellular_dataset/Timepoint_005_220518-ST_C03_s1.jpg"
# img_path = "images/test.jpg"
img = app.load_image(img_path)

In [ ]:
# Create objects
# mean_pix = MeanPixelValue()
mean_pix = MeanPixelValue(channels=2) # test RGB
# theremin = Theremin()
theremin = Theremin(np.linspace(220, 440, 64).tolist(), panning=[-1, 1]) # test multichannel
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

In [ ]:
app.features

In [ ]:
app.attach_feature(mean_pix)
app.features

In [ ]:
app.synths

In [ ]:
app.attach_synth(theremin)
app.synths

In [ ]:
app.mappers

In [ ]:
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
app.detach_mapper(pix2freq)
app.mappers

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)

In [ ]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
# detach a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# detach old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# NRT prototyping

In [6]:
import signalflow as sf
from IPython.display import Audio, display
import numpy as np

## Example from signalflow docs

In [2]:
# Create an AudioGraph with a dummy output device
graph = sf.AudioGraph(output_device=sf.AudioOut_Dummy(2))

# Create a buffer that will be used to store the audio output
buffer = sf.Buffer(2, graph.sample_rate * 4)

# Create a synthesis graph to render
freq = sf.SawLFO(1, 200, 400)
sine = sf.SineOscillator([freq, freq+10])
graph.play(sine)

# Render to the buffer. Non-real-time, so happens instantaneously.
# Note that the graph renders as many samples as needed to fill the buffer.
graph.render_to_buffer(buffer)

# Write the buffer contents to a file
buffer.save("output.wav")

# Finally, tear down the buffer
graph.destroy()

In [ ]:
display(Audio("output.wav"))

In [15]:
# Create an AudioGraph with a dummy output device
graph = sf.AudioGraph(output_device=sf.AudioOut_Dummy(2))

# Create a synthesis graph to render
freqs = [220, 220, 440, 440, 880, 880, 220, 220]
times = [2, 1, 2, 1, 2, 1, 3]

# Create a buffer that will be used to store the audio output
buffer = sf.Buffer(2, graph.sample_rate * int(sum(times)))

freq = sf.Envelope(freqs, times)
sine = sf.SineOscillator([freq, freq])
graph.play(sine)

# Render to the buffer. Non-real-time, so happens instantaneously.
# Note that the graph renders as many samples as needed to fill the buffer.
graph.render_to_buffer(buffer)

# Write the buffer contents to a file
buffer.save("output_2.wav")

# Finally, tear down the buffer
graph.destroy()

# Load the audio file into a widget for playback
display(Audio("output_2.wav"))

In [ ]:
graph = sf.AudioGraph()

test_buf = sf.Buffer(1, 48000)
test_player = sf.BufferPlayer(test_buf, loop=True)

graph.play(test_player)

new_buf = sf.Buffer(1, 48000)
new_buf.data[0, :] = np.linspace(0, 1, 48000)
test_player.set_buffer("buffer", new_buf)

In [54]:
graph.stop(test_player)

In [ ]:
graph.sample_rate

In [46]:
# test if a param buffer could be changed after the theremin is created
new_buf_sr = 10
new_freq_buffer = sf.Buffer(1, new_buf_sr)
new_freq_buffer.sample_rate = new_buf_sr
new_freq_buffer.data[0, :] = np.linspace(100, 1000, new_buf_sr)

In [ ]:
new_freq_buffer.sample_rate

In [ ]:
new_freq_buffer.data

In [ ]:
new_freq_buffer.sample_rate = 16000
new_freq_buffer.sample_rate, new_freq_buffer.duration

In [49]:
theremin.frequency_value.set_buffer("buffer", new_freq_buffer)

In [50]:
theremin.frequency_value.set_buffer("buffer", theremin.frequency_buffer)

In [ ]:
# horizontal scan test
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]

target_filename = "nrt_test_horizontal_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

In [ ]:
# vertical scan test
duration = 5
my_timeline = [
    (0, {
        "probe_width": 500,
        "probe_height": 1,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_y": 499
    })
]

target_filename = "nrt_test_vertical_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

# SynthGroup proto

In [1]:
import numpy as np
import signalflow as sf

In [ ]:
# create a graph
graph = sf.AudioGraph.get_shared_graph()
if graph is None:
    graph = sf.AudioGraph()
else:
    graph.destroy()
    graph = sf.AudioGraph()
graph.status

In [3]:
# create a patch with a sine oscillator and a frequency input
class SineTest(sf.Patch):
    def __init__(self, frequency=440):
        super().__init__()
        frequency = self.add_input("frequency", frequency)
        sine = sf.SineOscillator(frequency)
        stereo = sf.StereoPanner(sine, pan=0)
        self.set_output(stereo)

In [18]:
# create an instance
sine_test = SineTest()
graph.play(sine_test)

In [8]:
# create another instance
sine_test2 = SineTest()
graph.play(sine_test2)

In [9]:
# test setting the frequency
sine_test.set_input("frequency", 220)

In [19]:
graph.stop(sine_test)
graph.stop(sine_test2)

In [28]:
# create a patch that should have instances with different frequencies
class SineTestGroup(sf.Patch):
    def __init__(self, frequencies=[440, 440]):
        super().__init__()
        frequencies = self.add_input("frequencies", frequencies)
        sine = sf.SineOscillator(frequencies)
        mono = sf.ChannelMixer(1, sine)
        stereo = sf.StereoPanner(mono, pan=0)
        self.set_output(stereo)

In [ ]:
# create an instance
fundamental_freq = 220
freqs = [fundamental_freq * i for i in range(1, 5)]
print(freqs)
sine_test_group = SineTestGroup(freqs)
graph.play(sine_test_group)

In [30]:
graph.stop(sine_test_group)

In [38]:
# try the same but with input buffers
class SineTestGroup2(sf.Patch):
    def __init__(self, frequencies=[440, 440]):
        super().__init__()
        num_voices = len(frequencies)
        self.frequency_buffer = sf.Buffer(num_voices, 1)
        frequency_array = np.array(frequencies).reshape(num_voices, 1)
        self.frequency_buffer.data[:, :] = frequency_array
        self.frequency_reader = sf.BufferPlayer(self.frequency_buffer, loop=True)
        sine = sf.SineOscillator(self.frequency_reader)
        mono = sf.ChannelMixer(1, sine)
        stereo = sf.StereoPanner(mono, pan=0)
        self.set_output(stereo)

In [ ]:
# create an instance
fundamental_freq = 220
freqs = [fundamental_freq * i for i in range(1, 10)]
print(freqs)
sine_test_group2 = SineTestGroup2(freqs)
graph.play(sine_test_group2)

In [53]:
# try to modify the param buffer
sine_test_group2.frequency_buffer.data[0, :] = 230

In [54]:
graph.stop(sine_test_group2)

In [78]:
sine1 = sf.SineOscillator(220)
sine2 = sf.SineOscillator(330)

pansig1 = sf.SineOscillator(0.1)
pansig2 = sf.SineOscillator(0.11)

multisine = [sine1, sine2]
multipansig = [pansig1, pansig2]

panner = [sf.StereoPanner(sine, pan) for sine, pan in zip(multisine, multipansig)]

bus = sf.Bus(2)

for p in panner:
    bus.add_input(p)

# print(panner)

out = bus / len(panner)

graph.play(bus)

In [21]:
multisine = sf.SineOscillator([220, 300, 402])
multipansig = sf.SineOscillator([0.1, 0.11, 0.12])

n = multisine.num_output_channels
panner = [sf.StereoPanner(multisine[i] / n, multipansig[i]) for i in range(n)]

bus = sf.Bus(2)

for p in panner:
    bus.add_input(p)

graph.play(bus)

In [8]:
m = 16 # number of output channels in the mixer
multisine = sf.SineOscillator([220, 300, 402])
multipansig = (sf.SineOscillator([0.1, 0.11, 0.12]) + 1) / 2 * (m-1) # scale [-1, 1] to [0, m-1]

n = multisine.num_output_channels
panner = [sf.ChannelPanner(m, multisine[i] / n, multipansig[i]) for i in range(n)]

bus = sf.Bus(m)

for p in panner:
    bus.add_input(p)

# mix down to stereo for listening
out_stereo = sf.ChannelMixer(2, bus)

graph.play(out_stereo)

In [15]:
graph.start()

In [11]:
graph.stop(bus)

In [7]:
graph.stop(out_stereo)

In [17]:
graph.stop()

In [ ]:
print(len(panner))

In [8]:
class Mixer(sf.Patch):
    def __init__(self, input_sig, pan_sig, num_channels=2):
        super().__init__()
        assert input_sig.num_output_channels == pan_sig.num_output_channels
        n = input_sig.num_output_channels
        panner = [sf.ChannelPanner(num_channels, input_sig[i] / n, pan_sig[i]) for i in range(n)]
        _sum = sf.Sum(panner)
        self.set_output(_sum)

In [9]:
# test the mixer
m = 16
multisine = sf.SineOscillator([220, 300, 402])
multipansig = (sf.SineOscillator([0.1, 0.11, 0.12]) + 1) / 2 * (m-1) # scale [-1, 1] to [0, m-1]
mix_multi = Mixer(multisine, multipansig, num_channels=m)
mix_stereo = sf.ChannelMixer(2, mix_multi)

graph.play(mix_stereo)

In [11]:
graph.stop(mix_stereo)

In [ ]:
mix_multi.state

In [14]:
class HupMixer(sf.Patch):
    def __init__(self, input_sig, num_channels=5):
        super().__init__()
        n = input_sig.num_output_channels # e.g. 2
        output_x = np.linspace(0, n-1, num_channels) # e.g. [0, 0.25, 0.5, 0.75, 1]
        upmixed_list = []
        for i in range(num_channels - 1):
            output_i = output_x[i]
            a = input_sig[int(output_i)]
            b = input_sig[int(output_i) + 1]
            frac = float(output_i - int(output_i))
            interp = sf.WetDry(a, b, sf.Constant(frac))
            upmixed_list.append(interp)
        # add the last channel
        upmixed_list.append(input_sig[n-1])
        _out = sf.ChannelArray(upmixed_list)
        self.set_output(_out)

In [17]:
class UpMixer(sf.Patch):
    def __init__(self, input_sig, out_channels=5):
        super().__init__()
        n = input_sig.num_output_channels # e.g. 2
        output_x = np.linspace(0, n-1, out_channels) # e.g. [0, 0.25, 0.5, 0.75, 1]
        output_y = output_x * (out_channels - 1) # e.g. [0, 1, 2, 3, 4]
        upmixed_list = [sf.WetDry(input_sig[int(output_i)], input_sig[int(output_i) + 1], float(output_i - int(output_i))) for output_i in output_x[:-1]]
        upmixed_list.append(input_sig[n-1])
        expanded_list = [sf.ChannelPanner(out_channels, upmixed_list[i], float(output_y[i])) for i in range(out_channels)]
        _out = sf.Sum(expanded_list)
        self.set_output(_out)

In [14]:
test_in = sf.SineOscillator([220, 300])
wtf = UpMixer(test_in, out_channels=5)
wtf_stereo = sf.ChannelMixer(2, wtf)
graph.play(wtf_stereo)

In [ ]:
wtf.state

In [15]:
graph.stop(wtf)

In [ ]:
b = sf.Buffer(3, 1)
b.data[:, :] = np.ones_like(b.data) * np.linspace(0, 1, 3).reshape(3, 1) # [0, 0.5, 1]
b_player = sf.BufferPlayer(b, loop=True)
b_upmixed = UpMixer(b_player, out_channels=5)
# b_upmixed = sf.ChannelMixer(5, b_player, True)
mixdown = sf.ChannelMixer(2, b_upmixed)
graph.play(mixdown)

In [20]:
graph.stop(mixdown)

In [ ]:
b_upmixed.state

In [8]:
one = sf.Constant(1)
two = sf.Constant(2)

interp = sf.WetDry(one, two, 0.5) # 1.5

In [12]:
interp.stop()

In [ ]:
interp.output_buffer[0, -1]

In [3]:
# test multiplication with different channels
a = sf.Buffer(5, 1)
a.data[:, :] = np.ones_like(a.data)
a_player = sf.BufferPlayer(a, loop=True)

b = sf.Buffer(3, 1)
b.data[:, :] = np.ones_like(b.data) * np.linspace(0, 1, 3).reshape(3, 1) # [0, 0.5, 1]
b_player = sf.BufferPlayer(b, loop=True)

c = a_player * b_player # it will tile b to [0, 0.5, 1, 0, 0.5]

d = sf.ChannelMixer(2, c)

graph.play(d)


In [6]:
graph.stop(d)

In [ ]:
# try to resample instead of tiling
a = sf.Buffer(5, 1)
a.data[:, :] = np.ones_like(a.data)
a_player = sf.BufferPlayer(a, loop=True)

b = sf.Buffer(3, 1)
b.data[:, :] = np.ones_like(b.data) * np.linspace(0, 1, 3).reshape(3, 1) # [0, 0.5, 1]
b_player = sf.BufferPlayer(b, loop=True)

b_upmixed = Mixer(b_player, [], num_channels=5)

c = a_player * b_player # it will tile b to [0, 0.5, 1, 0, 0.5]

d = sf.ChannelMixer(2, c)

graph.play(d)

In [ ]:
c.num_output_channels

In [ ]:
c.output_buffer[:, -1]

In [ ]:
a.data

In [ ]:
b.data

In [23]:
def resize_interp(
    input: np.ndarray,
    size: int,
) -> np.ndarray:
    """
    Resize an array. Uses linear interpolation.

    Args:
        input (np.ndarray): Array to resize.
        size (int): The new size of the array.

    Returns:
        np.ndarray: The resized array.
    """
    # create x axis for input
    input_x = np.arange(0, len(input))
    # create array with sampling indices
    output_x = np.linspace(0, len(input_x)-1, size)
    # interpolate
    return np.interp(output_x, input_x, input)

In [ ]:
a = resize_interp([0, 5, 2], 11)
# convert to list
a.tolist()


In [31]:
# helper function to broadcast and interpolate all param lists to the same length
def broadcast_params(*param_lists):
    # if an input list is just a single value, convert it to a list
    param_lists = [p if isinstance(p, list) else [p] for p in param_lists]
    max_len = max([len(p) for p in param_lists])
    broadcasted_params = []
    for plist in param_lists:
        if len(plist) < max_len:
            # interpolate
            plist = resize_interp(plist, max_len).tolist()
        broadcasted_params.append(plist)
    return broadcasted_params

In [ ]:
test_freqs = [220, 330, 440]
test_freqs = 220
test_pans = 0.1
test_amps = [0.5, 0.7, 0.9, 1.0]
test_amps = 0.7

new_freqs, new_pans, new_amps = broadcast_params(test_freqs, test_pans, test_amps)

new_freqs, new_pans, new_amps

In [35]:
testy = sf.SineOscillator([220])
stereo = sf.StereoPanner(testy, 0)
graph.play(stereo)

In [36]:
graph.stop(stereo)

In [37]:
test_list = [220, 330, 440]
testbuf = sf.Buffer(len(test_list), 1)
testbuf.data[:, :] = np.array(test_list).reshape(len(test_list), 1)

In [ ]:
import numpy as np

a = np.random.rand(50, 50, 3)
a.shape

In [ ]:
b = np.mean(a, axis=(0, 1))
b.shape

In [ ]:
a = np.random.rand(3)
b = np.random.rand(3)

a if a == b else b

In [ ]:
if a.all() == b.all():
    print("yes")

In [ ]:
a = np.zeros((2, 100))
b = np.ones((2, 1))

a[:, 0] = b[:, 0]

In [ ]:
a